In [1]:
#!pip install numpy

In [2]:
#!pip install matplotlib

In [3]:
#!pip install Cython

In [4]:
#!pip install pandas

In [5]:
#Si se desea graficar
#conda install -c conda-forge cartopy

In [6]:
#!pip install netcdf4

In [7]:
#!pip install tensorflow

In [8]:
#!pip install -U scikit-learn

In [9]:
#Manejo de Datos
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Imagenes satelitales
#import cartopy.crs as ccrs
#import cartopy.feature as cf
from netCDF4 import Dataset, num2date

#Machine learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

#Librerias estandar (Extras)
import re
import os
import time
import random

In [10]:
#conda uninstall cudnn

In [11]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10576170694722177446
xla_global_id: -1
]
Num GPUs Available:  0


In [12]:
#El modelo solo considera en input_shape(x,x,1), el 1 se puede cambiar para abarcar mas canales de imagenes satelitales
def crearModelo(dimTime,W,H,dimCanal, output):
    
    print(f"Se creo un modelo con input ({dimTime}, {W},{H}, {dimCanal}) y output({output})")
    model = models.Sequential()
    model.add(layers.Conv3D(32, (3, 3,3), activation='relu', input_shape=(dimTime, W, H, dimCanal)))
    model.add(layers.MaxPooling3D((2, 2,2)))
    #model.add(layers.Conv3D(128, (3, 3,3), activation='relu'))
    #model.add(layers.MaxPooling3D((2, 2,2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(output))
    
    print(model.summary())
    return model


In [13]:
def getMatriz(filename, origen, margen):    
    
    try:
        ds = Dataset(filename)
    except:
        print("No se pudo leer el archivo")
        print(filename)
        return -1

    #El ancho y alto sera el margen que se dara desde el punto de origen (estacion)
    #Esta en decimales (1 decimal == 100Km) - (Temporal)
    alto= margen[0]
    ancho= margen[1]
    
    
    #Obteine los datos de la imagen satelital
    field = ds.variables['CMI'][:].data.astype(np.float32)/100.0      
    lon = ds.variables['longitude'][:].data
    lat = ds.variables['latitude'][:].data    

    #Se define el margen para recortar la imagen satelital
    maxLon=origen[0]+ancho
    minLon=origen[0]-ancho
    maxLat=origen[1]+alto
    minLat=origen[1]-alto

    #Booleanos que ayudarán a buscar el margen
    altoMin = False
    altoMax = False


    #Inicializamos los "indices"
    lom = 0
    loM = 0
    lam = 0
    laM = 0

    """
    Tener en cuenta que el arreglo de longitudes (lon) esta ordenado de manera creciente,
    mientras que el de latitudes (lat) esta de manera decreciente
    """    
    for i in range(0,len(lon)):
        if lon[i]>=minLon and not altoMin:
            altoMin = True
            lom = i
        if lon[i]<=maxLon:
            loM = i

    for j in range(0,len(lat)):
        if lat[j]>=minLat:    
            laM = j
        if lat[j]<=maxLat and not altoMax:
            altoMax = True
            lam = j  
            
    return field[lam:laM,lom:loM]

In [14]:
#Se le da un tensor de 4 dimensiones
#[0] =  dato de precipitacion
#[1] = Punto de la estacion (Longitud)
#[2] = Punto de la estacion (Latitud)
#[3] = Fecha (año-mes-dia-hora)

#Devuelve x,y
#X = Dato de precipitacion
#Y = np.Array de las matrices de colores de cada producto en products (C08,C07 o C13)
def leerImagenArea(tensor, umbral, path_base,margen,products, imprimir=0): 
    
    """
    -----------------------------------------------------------------------------------    
    !!!!!VERIFICAR QUE LA HORA DE LA IMAGEN SATELITAL SEA IGUAL A LA HORA PERU!!!!!!!!
    -----------------------------------------------------------------------------------
    
    Los archivos se deben encontrar en carpetas ordenadas : ../GOES/{producto}/{año}/{mes}/{ARCHIVO}.nc
    ARCHIVO = G16_{producto}_Cyl_{año}{mes}{dia}-{hora}00.nc'
    
    EJEMPLO : path_base + GOES/C8/2019/02/G16_C08_Cyl_20190210-1600.nc
    """
    
    #Se define por defecto el path base - (Temporal)    
    try:
        #Fecha = 2019 01 05 22
        fecha = str(tensor.numpy()[3].decode('UTF-8'))
        year,month,day,hour = fecha.split('-')        
   
    except:
        print("No se pudo leer la fecha")
        print(tensor.numpy()[3].decode('UTF-8'))
        return -1
    
    
    origen = [float(tensor.numpy()[1].decode('UTF-8')),float(tensor.numpy()[2].decode('UTF-8'))] 
    
    #Se define el producto 
    mapaArrays = []
    
    times = [['15','20'],['30','30'],['45','40'],['00','00']]
    dimTime = len(times)
    
    start_time = time.time()
    for i in range(dimTime):
        productMatriz  =  []
        for product in products:
            j = 1 if product == 'C02' else 0   
            ############TODO#"###################################
            hourTemp  = int(hour)-((i//(dimTime-1)==0))
            filename = f'{path_base}{product}/{year}/{month}/G16_{product}_Cyl_{year}{month}{day}-{hourTemp:02d}{times[i][j]}.nc'
            #print(filename)
            mProduct = getMatriz(filename, origen, margen)            
            productMatriz.append(mProduct)
        mapaArrays.append(np.dstack(productMatriz))
        
    mapaArrays  = np.array(mapaArrays) 
    dato = float(tensor.numpy()[0].decode('UTF-8'))
    if imprimir:
        print(f"Tiempo tomado en obtener matrices de un dato para {len(products)} productos: %.2fs" % (time.time() - start_time))
    
    dato = 1 if dato>umbral else 0    
    if len(products) == 1:
        return mapaArrays, dato    
    
    return mapaArrays, dato
       

In [15]:
#Devuelve una lista con lo indices que no se encontraron lso archivos y el producto
#Servira para ver si se teinen todas las imagenes necesarias para el entrenamiento
def comprobarFile(df,products,path_base):     
    start_time = time.time()
    no_index = []
    no_product = []
    no_fecha = []
    for i in df.index:       
        year,month,day,hour = df['fecha'][i].split('-')
        tmpProduct = []        
        for p in products:
            timeDim = ['15','30','45']        
            if p == 'C02':
                timeDim = ['20','30','40']
            
            filename = f'{path_base}/{p}/{year}/{month}/G16_{p}_Cyl_{year}{month}{day}-{hour}00.nc'       
            existe = os.path.exists(filename)
            if not existe:
                tmpProduct.append((p,td))
                                      
            for td in timeDim:
                filename = f'{path_base}/{p}/{year}/{month}/G16_{p}_Cyl_{year}{month}{day}-{int(hour)-1 }{td}.nc'       
                existe = os.path.exists(filename)
                if not existe:
                    tmpProduct.append((p,td))
        if len(tmpProduct)>0:
            no_index.append(i)
            no_fecha.append(df['fecha'][i])                
            no_product.append(tmpProduct)
                               
       
    df2 = df.drop(index=no_index)
    print(f'{len(no_index)} datos eliminados: No se encontraron los archivos de imagenes satelitales')
    print("Tiempo tomado en verificar datos: %.2fs" % (time.time() - start_time))
    return df2 , (no_fecha,no_product)

In [16]:
def obtenerDatos(filename):
    start_time = time.time()
    pdata = pd.read_csv(filename) 
    
    #Seleccionamos solo las columnas necesarias : precipitacion, Estacion (Longitud), Estacion (Latitud), Fecha (año-mes-dia-hora)
    pdataX = pdata.loc[:, ['dato','longitud', 'latitud', 'fecha']]

    #Quitamos los valores NA
    pdataX = pdataX[pdataX['dato'].notna()]

    #Definimos un solo tipo (str) pora asi poder convertirlo a tensor
    pdataX = pdataX.astype({"dato":str,"longitud":str, "latitud":  str, "fecha": str})
                
    #Barajeamos los datos
    pdataX = shuffle(pdataX)
    
    print(f'{len(pdataX)} datos leidos')
    print("Tiempo tomado en leer datos: %.2fs" % (time.time() - start_time))
    return pdataX

In [17]:
def xyDataset(dataset,umbral, path_base,margen,products):
    x = []
    y = []
    i,j = 0.0 , []    
    for dato in dataset:       
        i,j =  leerImagenArea(dato,umbral, path_base,margen,products)
        x.append(i)
        y.append(j)
    x = np.asarray(x)
    y = np.asarray(y)
    return x,y
    

In [18]:
#@tf.function
def train_step(x,y,model,optimizer,loss_fn,train_acc_metric):    
    
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_acc_metric.update_state(y, logits)
    return loss_value

In [19]:
#@tf.function
def test_step(x,y,model,val_acc_metric):      
    val_logits = model(x, training=False)
    val_acc_metric.update_state(y, val_logits)

In [20]:
def entrenamiento(datasetList,umbral,model,path_base,margen,products, batch_size,train_size,epocas=2,imprimir=0):  

    #Dividmos el dataset (Entrenamient - Validacion)
    train_dataset = tf.data.Dataset.from_tensor_slices(datasetList[:train_size])           
    val_dataset = tf.data.Dataset.from_tensor_slices(datasetList[train_size:])

    #Divimos en batchs los datasets
    train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)
    val_dataset = val_dataset.batch(batch_size)
    

    #Definimos Variables del modelo (optmizador, funcion de loss, metricas, etc)
    optimizer = keras.optimizers.Adam(learning_rate=1e-3)        
    loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
    val_acc_metric = keras.metrics.SparseCategoricalAccuracy()
        
    #Entrenamos el modelo
    for epoch in range(epocas):    
        print("\nComienzo de la epoca %d" % (epoch,))
        start_time = time.time()
        
        for step, (datos) in enumerate(train_dataset):            
            #Obtenmos el verdadero dataset (valor, matriz) del batch X
            start_time_data = time.time()
            x_train_batch, y_train_batch =  xyDataset(datos,umbral, path_base,margen,products) 
            if imprimir:
                print(f"Tiempo tomado para leer un batch de entrenamiento ({batch_size} datos): %.2fs" % (time.time() - start_time_data))
            
            #Se obtiene el valor de perdida para el batch X
            start_time_train = time.time()
            loss_value = train_step(x_train_batch,y_train_batch,model,optimizer,loss_fn,train_acc_metric)
            if imprimir:
                print(f"Tiempo tomado para entrenar un batch ({batch_size} datos): %.2fs" % (time.time() - start_time_train))            
                
            #Log every 200 batches.
            if step % 5 == 0:
                print(
                    "Training loss (for one batch) at step %d: %.4f"
                    % (step, float(loss_value))
                )
                print("Seen so far: %d samples" % ((step + 1) * batch_size))
        
        #Imprimimos y reiniciamos las metricas para una epoca
        train_acc = train_acc_metric.result()
        print("Training acc over epoch: %.4f" % (float(train_acc),))        
        train_acc_metric.reset_states()

        #Usamos el dataset de validacion para la validacion
        for (datos) in val_dataset:
            #Verdadero dataset
            start_time_data2 = time.time()
            x_val_batch, y_val_batch =  xyDataset(datos,umbral, path_base,margen,products)
            if imprimir:
                print(f"Tiempo tomado para leer un batch de evaluacion ({batch_size} datos): %.2fs" % (time.time() - start_time_data2))
                
            start_time_evaluate = time.time()
            #Evaluamos
            test_step(x_val_batch, y_val_batch, model,val_acc_metric)
            if imprimir:
                print(f"Tiempo tomado para evaluar un batch ({batch_size} datos): %.2fs" % (time.time() - start_time_evaluate))

        #Imprimimos y reinciamos
        val_acc = val_acc_metric.result()
        print("Validation acc: %.4f" % (float(val_acc),))    
        val_acc_metric.reset_states()
        print("Tiempo tomado en entrenar una epoca: %.2fs" % (time.time() - start_time))

In [21]:
#Path_base debe ser completo, se usará para comprobar si existen las imagenes satelitales descargadas
path_base = 'C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/GOES/'

#El margen servira para recortar la imagen [alto, ancho] desde el punto de origen (estacion), esta en decimales
margen = [1,1]

#POR EL MOMENTO el batch_size debe poder dividirse entre la cantidad total del dataset (no residuo) 
batch_size = 100

#Representa el output del modelo (2 clases)
#1 = Precipitacion extrema
#0 = Precipitacion normal
dimOutput = 2

In [22]:
#Leemos los datos del archivo
#Archivo de prueba contiene datos del 2019 del mes 01 y 02
dfOrignial = obtenerDatos('pruebasV2.csv')

336720 datos leidos
Tiempo tomado en leer datos: 0.76s


In [23]:
##################################################
####Se entrenara con 2 products (C08,C07)#####
##################################################

In [24]:
#Productos de las imagenes satelitales (C08, C07 o C13, C02)
products = ['C08','C07']

#Comprobamos si existen las imagenes/produtos por cada dato,
#caso contrario los borra de la lista
dfVerificado, (no_i,no_p) = comprobarFile(dfOrignial,products,path_base)

302846 datos eliminados: No se encontraron los archivos de imagenes satelitales
Tiempo tomado en verificar datos: 60.91s


In [25]:
#Seleccionamos algunos para las pruebas
df = dfVerificado[0:1000]
datasetList = df.values.tolist()

#-Visualizacion
print(len(datasetList))
print(datasetList[0])

1000
['0.0', '-72.29832', '-13.98923', '2019-01-28-15']


In [38]:
#Creamos el modelo
dimOutput = 2
tempTensor =  tf.constant(datasetList[0])
imagenT, datoT = leerImagenArea(tempTensor, 5,path_base,margen,products)
modelo = crearModelo(imagenT.shape[0],imagenT.shape[1],imagenT.shape[2],imagenT.shape[3],dimOutput)

Se creo un modelo con input (4, 110,110, 2) y output(2)
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_2 (Conv3D)           (None, 2, 108, 108, 32)   1760      
                                                                 
 max_pooling3d_2 (MaxPooling  (None, 1, 54, 54, 32)    0         
 3D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 93312)             0         
                                                                 
 dense_4 (Dense)             (None, 64)                5972032   
                                                                 
 dense_5 (Dense)             (None, 2)                 130       
                                                                 
Total params: 5,973,922
Trainable params: 5,973,922
Non-trainabl

In [27]:
#El Umbral esta en mm/h, igual que el dataset. Si supera este umbral se considera 1 (Extremo) sino 0 (no extremo)
umbral = 2.0

train_size = int(len(datasetList)*0.8)
epocas = 1

#Entrenamos con products >= 2 el dataset              
entrenamiento(datasetList,umbral,modelo,path_base,margen,products, batch_size,train_size, epocas)


Comienzo de la epoca 0
Training loss (for one batch) at step 0: 182.7739
Seen so far: 100 samples
Training loss (for one batch) at step 5: 127.1277
Seen so far: 600 samples
Training acc over epoch: 0.8400
Validation acc: 0.9750
Tiempo tomado en entrenar una epoca: 144.92s


In [28]:
##################################################
####Se entrenara con solo 1 product (C08)#####
##################################################

In [29]:
#Productos de las imagenes satelitales (C08, C07 o C13, C02)
_products = ['C08']

#Comprobamos si existen las imagenes/produtos por cada dato,
#caso contrario los borra de la lista
_dfVerificado, (_no_i,_no_p) = comprobarFile(dfOrignial,_products,path_base)

167400 datos eliminados: No se encontraron los archivos de imagenes satelitales
Tiempo tomado en verificar datos: 32.52s


In [30]:
#Seleccionamos algunos para las pruebas
_df = _dfVerificado[0:1000]
_datasetList = _df.values.tolist()

#-Visualizacion
print(len(_datasetList))
print(_datasetList[0])

1000
['0.0', '-78.82367', '-7.47987', '2019-02-05-18']


In [31]:
#Creamos el modelo
_tempTensor =  tf.constant(datasetList[0])
_imagenT, _datoT = leerImagenArea(_tempTensor, 5,path_base,margen,_products)
_modelo = crearModelo(_imagenT.shape[0],_imagenT.shape[1],_imagenT.shape[2],_imagenT.shape[3],dimOutput)

Se creo un modelo con input (4, 110,110, 1) y output(2)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_1 (Conv3D)           (None, 2, 108, 108, 32)   896       
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 1, 54, 54, 32)    0         
 3D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 93312)             0         
                                                                 
 dense_2 (Dense)             (None, 64)                5972032   
                                                                 
 dense_3 (Dense)             (None, 2)                 130       
                                                                 
Total params: 5,973,058
Trainable params: 5,973,058
Non-trainabl

In [32]:
#El Umbral esta en mm/h, igual que el dataset. Si supera este umbral se considera 1 (Extremo) sino 0 (no extremo)
umbral = 2.0

train_size = int(len(datasetList)*0.8)
epocas = 1

#Entrenamos con products 1 el dataset              
entrenamiento(_datasetList,umbral,_modelo,path_base,margen,_products, batch_size,train_size, epocas)


Comienzo de la epoca 0
Training loss (for one batch) at step 0: 16.5008
Seen so far: 100 samples
Training loss (for one batch) at step 5: 148.0173
Seen so far: 600 samples
Training acc over epoch: 0.8550
Validation acc: 0.9900
Tiempo tomado en entrenar una epoca: 92.22s


In [33]:
#########################
#Estadisticas de tiempo#
#########################

In [34]:
tempTensor = tf.constant(datasetList[0])
print("Tiempo para leer matriz de un solo producto")
x,y = leerImagenArea(tempTensor, 5.0, path_base,margen,['C08'], 1) 

Tiempo para leer matriz de un solo producto
Tiempo tomado en obtener matrices para un dota: 0.06s


In [35]:
print("Tiempo para leer matriz 2 productos")
x,y = leerImagenArea(tempTensor, 5.0, path_base,margen,['C08','C07'], 1) 

Tiempo para leer matriz de un solo producto
Tiempo tomado en obtener matrices para un dota: 0.09s


In [36]:
#Entrenamos con products 1 el dataset              
entrenamiento(_datasetList,umbral,_modelo,path_base,margen,_products, batch_size,train_size, epocas,1)


Comienzo de la epoca 0
Tiempo tomado para leer un batch de entrenamiento (100 datos): 5.89s
Tiempo tomado para entrenar un batch (100 datos): 3.22s
Training loss (for one batch) at step 0: 92.0272
Seen so far: 100 samples
Tiempo tomado para leer un batch de entrenamiento (100 datos): 5.74s
Tiempo tomado para entrenar un batch (100 datos): 3.19s
Tiempo tomado para leer un batch de entrenamiento (100 datos): 5.91s
Tiempo tomado para entrenar un batch (100 datos): 3.21s
Tiempo tomado para leer un batch de entrenamiento (100 datos): 5.87s
Tiempo tomado para entrenar un batch (100 datos): 3.20s
Tiempo tomado para leer un batch de entrenamiento (100 datos): 5.94s
Tiempo tomado para entrenar un batch (100 datos): 3.19s
Tiempo tomado para leer un batch de entrenamiento (100 datos): 5.88s
Tiempo tomado para entrenar un batch (100 datos): 3.22s
Training loss (for one batch) at step 5: 0.0000
Seen so far: 600 samples
Tiempo tomado para leer un batch de entrenamiento (100 datos): 5.89s
Tiempo tom

In [39]:
#Entrenamos con products 2 el dataset              
entrenamiento(datasetList,umbral,modelo,path_base,margen,products, batch_size,train_size, epocas,1)


Comienzo de la epoca 0
Tiempo tomado para leer un batch de entrenamiento (100 datos): 11.72s
Tiempo tomado para entrenar un batch (100 datos): 3.28s
Training loss (for one batch) at step 0: 1.2648
Seen so far: 100 samples
Tiempo tomado para leer un batch de entrenamiento (100 datos): 11.80s
Tiempo tomado para entrenar un batch (100 datos): 3.32s
Tiempo tomado para leer un batch de entrenamiento (100 datos): 11.52s
Tiempo tomado para entrenar un batch (100 datos): 3.29s
Tiempo tomado para leer un batch de entrenamiento (100 datos): 11.77s
Tiempo tomado para entrenar un batch (100 datos): 3.29s
Tiempo tomado para leer un batch de entrenamiento (100 datos): 11.74s
Tiempo tomado para entrenar un batch (100 datos): 3.26s
Tiempo tomado para leer un batch de entrenamiento (100 datos): 11.59s
Tiempo tomado para entrenar un batch (100 datos): 3.29s
Training loss (for one batch) at step 5: 224.6156
Seen so far: 600 samples
Tiempo tomado para leer un batch de entrenamiento (100 datos): 11.65s
Ti